# Projet Optimisation de portefeuille - EDA

In [20]:
import pandas as pd
import numpy as np
import os
import glob
import json
import sys

project_root = os.path.dirname(os.path.abspath(''))
raw_data = os.path.join(project_root, 'data', 'raw')
processed_data = os.path.join(project_root, 'data', 'processed')

if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_prep import all_sectors

### Création et chargement des données

In [21]:
all_sectors()
df = all_sectors()

df.head()

Fichier all_sectors.csv sauvegardé
Nombre total de lignes : 2516
Fichier all_sectors.csv sauvegardé
Nombre total de lignes : 2516


,LIN,APD,ECL,SHW,PPG,MLM,VMC,NUE,STLD,FCX,...,META,DIS,NFLX,VZ,T,TMUS,CMCSA,CHTR,EA,TTWO
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,106.324860,103.627495,92.699158,78.926811,94.352951,101.658020,61.198994,37.831028,15.793311,20.731445,...,77.969345,85.943512,4.984857,26.573195,11.657843,26.374689,22.256119,166.429993,45.571400,28.100000
2015-01-05,103.518425,100.314102,90.628548,78.692230,91.918037,97.512856,59.508686,36.387573,14.840947,19.499002,...,76.717056,84.687607,4.731143,26.352510,11.547701,26.017233,21.802221,160.500000,45.279724,27.620001
2015-01-06,102.192963,100.278473,87.918022,78.454651,91.691826,96.577431,59.371635,35.862659,14.412383,19.833527,...,75.683441,84.238380,4.650143,26.618467,11.564909,25.959267,21.517811,156.309998,44.686626,27.930000
2015-01-07,103.338425,101.361572,89.206612,79.792953,92.099030,98.833412,60.495476,36.194599,14.618734,20.124029,...,75.683441,85.100136,4.674286,26.446692,11.632340,27.292488,21.498337,156.789993,45.513077,28.100000
2015-01-08,105.040314,103.705894,91.197296,81.976372,94.550400,102.630142,62.386791,37.074551,15.126663,20.572992,...,77.700996,85.980194,4.778000,27.013535,11.748068,28.036394,21.965862,162.940002,46.796509,28.459999


#### Calcul des rendements journaliers
Formule : (Prix_jour_J - Prix_jour_J-1) / Prix_jour_J-1

In [22]:
returns = df.pct_change()
returns.drop(returns.index[0], inplace=True)

missing_values = returns.isnull().sum()
non_zero_missing = missing_values[missing_values > 0]
if not non_zero_missing.empty:
    print(f"Nombre de valeurs manquantes :\n{non_zero_missing}")

Nombre de valeurs manquantes :
OTIS    1311
SNOW    1436
SHOP      95
CRWD    1117
ZS       806
MDB      705
KHC      126
dtype: int64


In [23]:
# On nettoie les colonnes qui contiennent des NaN, cela va retirer KHC, OTIS, SNOW, etc. mais garder tout l'historique 2015-2024
# on fait ça car on ne doit avoir aucune valeur manquante pour la suite
returns_clean = returns.dropna(axis=1)

print(f"Avant nettoyage : {returns.shape[1]} actifs")
print(f"Après nettoyage : {returns_clean.shape[1]} actifs")
print(f"Valeurs manquantes restantes : {returns_clean.isnull().sum().sum()}")

Avant nettoyage : 197 actifs
Après nettoyage : 190 actifs
Valeurs manquantes restantes : 0


Export

In [24]:
returns_clean.to_csv(os.path.join(processed_data, 'daily_returns.csv'))